In [44]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' #this helps to full output and not only the last lines of putput

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.simplefilter('ignore')

In [45]:
df = pd.read_csv(r"D:\IIIT_delhi\2_Sem\Information_Retrieval\28_Baseline\customers_rating.csv")

In [46]:
df.head()

,Unnamed: 0,UserId,ProductId,Rating,Timestamp
0,0,A39HTATAQ9V7YF,0205616461,5.0,1369699200
1,1,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
2,2,A1Z513UWSAAO0F,0558925278,5.0,1404691200
3,3,A1WMRR494NWEWV,0733001998,4.0,1382572800
4,4,A3IAAVS479H7M7,0737104473,1.0,1274227200


In [47]:
df_copy = df.copy()

In [48]:
df_copy=df_copy.drop('Unnamed: 0', axis=1)

In [49]:
df_copy=df_copy.drop('Timestamp', axis=1)

In [50]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   UserId     100000 non-null  object 
 1   ProductId  100000 non-null  object 
 2   Rating     100000 non-null  float64
dtypes: float64(1), object(2)
memory usage: 2.3+ MB


In [51]:
df_copy.head()

,UserId,ProductId,Rating
0,A39HTATAQ9V7YF,0205616461,5.0
1,A3JM6GV9MNOF9X,0558925278,3.0
2,A1Z513UWSAAO0F,0558925278,5.0
3,A1WMRR494NWEWV,0733001998,4.0
4,A3IAAVS479H7M7,0737104473,1.0


## Popularity Based Recommendation System 

In [52]:
#Counting the number of overall rating of all products and how many users have given the rating 
df_copy.groupby('ProductId').count()

,UserId,Rating
ProductId,,
0205616461,1,1
0558925278,2,2
0733001998,1,1
0737104473,1,1
0762451459,1,1
...,...,...
B0002VNKRM,17,17
B0002VNKSG,5,5
B0002VNLG2,2,2


In [53]:
num_rating_df = df_copy.groupby('ProductId').count()['Rating'].reset_index()
num_rating_df.rename(columns={'Rating':'num_rating'},inplace=True)
num_rating_df

,ProductId,num_rating
0,0205616461,1
1,0558925278,2
2,0733001998,1
3,0737104473,1
4,0762451459,1
...,...,...
6379,B0002VNKRM,17
6380,B0002VNKSG,5
6381,B0002VNLG2,2
6382,B0002VNZS6,2


In [54]:
#Average rating of all products.
avg_rating_df = df_copy.groupby('ProductId').mean()['Rating'].reset_index()
avg_rating_df.rename(columns={'Rating':'avg_rating'},inplace=True)
avg_rating_df

,ProductId,avg_rating
0,0205616461,5.000000
1,0558925278,4.000000
2,0733001998,4.000000
3,0737104473,1.000000
4,0762451459,5.000000
...,...,...
6379,B0002VNKRM,4.294118
6380,B0002VNKSG,5.000000
6381,B0002VNLG2,4.000000
6382,B0002VNZS6,3.000000


In [55]:
popular_df = num_rating_df.merge(avg_rating_df,on='ProductId')
popular_df

,ProductId,num_rating,avg_rating
0,0205616461,1,5.000000
1,0558925278,2,4.000000
2,0733001998,1,4.000000
3,0737104473,1,1.000000
4,0762451459,1,5.000000
...,...,...,...
6379,B0002VNKRM,17,4.294118
6380,B0002VNKSG,5,5.000000
6381,B0002VNLG2,2,4.000000
6382,B0002VNZS6,2,3.000000


In [56]:
#We will suggest only those products to users which have recieved number of ratings more than 5
popular_df[popular_df['num_rating']>=5]

,ProductId,num_rating,avg_rating
19,1304351475,16,3.562500
22,1304482685,6,3.833333
50,1403790965,11,4.909091
51,1412759676,6,4.833333
62,3227001381,25,4.560000
...,...,...,...
6377,B0002VNKPY,17,3.882353
6378,B0002VNKQS,9,3.222222
6379,B0002VNKRM,17,4.294118
6380,B0002VNKSG,5,5.000000


In [57]:
popular_df[popular_df['num_rating']>=5].sort_values('avg_rating',ascending=False)

,ProductId,num_rating,avg_rating
1646,B000066B1B,14,5.000000
4663,B00021VU8S,5,5.000000
2967,B00014H8L4,5,5.000000
6212,B0002SH5E4,5,5.000000
2983,B00014IXAY,5,5.000000
...,...,...,...
5705,B0002FCA92,6,1.833333
1579,B00005YU0D,5,1.800000
5911,B0002JBU1M,5,1.800000
1393,B00005CDRP,6,1.166667


In [58]:
#Displaying top 5 products which have highest average ratings
popular_df = popular_df[popular_df['num_rating']>=5].sort_values('avg_rating',ascending=False).head(5)
popular_df

,ProductId,num_rating,avg_rating
1646,B000066B1B,14,5.0
4663,B00021VU8S,5,5.0
2967,B00014H8L4,5,5.0
6212,B0002SH5E4,5,5.0
2983,B00014IXAY,5,5.0


## Content based recommendation system

In [59]:
#Grouping data based on user ID and finding number of ratings given by each user
unum_rating_df = df_copy.groupby('UserId').count()['Rating'].reset_index()
unum_rating_df.rename(columns={'Rating':'unum_rating'},inplace=True)
unum_rating_df

,UserId,unum_rating
0,A00205921JHJK5X9LNP42,1
1,A00473363TJ8YSZ3YAGG9,2
2,A00700212KB3K0MVESPIY,1
3,A0081289HG0BXFQJQUWW,1
4,A01247753D6GFZD87MUV8,1
...,...,...
91651,AZZTJQ7CQZUD8,1
91652,AZZVCBG5G4EV8,1
91653,AZZWJ3LICUEKJ,1
91654,AZZWPNME0GQZ2,1


In [60]:
#consider only those users which have been given number of ratings more than or equal to 2
x = df_copy.groupby('UserId').count()['Rating']>1
good_users = x[x].index
good_users

Index(['A00473363TJ8YSZ3YAGG9', 'A0733801U845JJGV4H8V', 'A0908131Z7BWYSMRQ16T',
       'A099766128UI0NCS98N1E', 'A1008GFLTBL76H', 'A100V5X97DUOXV',
       'A100VQNP6I54HS', 'A1016DX7QKYBDC', 'A103D23F4SGCJY', 'A103U3KR4L2ZXT',
       ...
       'AZTX0RHQ1O9CB', 'AZTX1196S2WR9', 'AZU3L6HCEFG7B', 'AZUI6YY673GW5',
       'AZVYUXWLR63AZ', 'AZW9UD7Z7J0WL', 'AZWXG6KBXXC2N', 'AZZHB6U54UDYW',
       'AZZNK89PXD006', 'AZZQXL8VDCFTV'],
      dtype='object', name='UserId', length=6014)

In [61]:
filtered_rating = df_copy[df_copy['UserId'].isin(good_users)]
filtered_rating

,UserId,ProductId,Rating
4,A3IAAVS479H7M7,0737104473,1.0
5,AKJHHD5VEH7VG,0762451459,5.0
6,A1BG8QW55XHN6U,1304139212,5.0
7,A22VW0P4VZHDE3,1304139220,5.0
10,A1BG8QW55XHN6U,130414643X,5.0
...,...,...,...
99742,A2P5CDZF56NLVL,B0002VNKRM,5.0
99746,A1ASSKIYJ4Z9K8,B0002VNKSG,5.0
99828,A6WV0OPKDYP2F,B0002VQ0WO,4.0
99848,A70G84Z95RU3N,B0002VQ0WO,5.0


In [62]:
#Count the number of ratings for each product
filtered_rating.groupby('ProductId').count()['Rating']

ProductId
0737104473    1
0762451459    1
1304139212    1
1304139220    1
130414643X    1
             ..
B0002VNKPY    3
B0002VNKQS    1
B0002VNKRM    7
B0002VNKSG    1
B0002VQ0WO    3
Name: Rating, Length: 3091, dtype: int64

In [63]:
#Display all the products which have recieved more than 1 ratings
y = filtered_rating.groupby('ProductId').count()['Rating'] >= 1
famous_resto = y[y].index
famous_resto

Index(['0737104473', '0762451459', '1304139212', '1304139220', '130414643X',
       '1304146537', '130414674X', '1304168522', '1304174778', '1304174867',
       ...
       'B0002VHBTU', 'B0002VHOBK', 'B0002VJIH8', 'B0002VJTGS', 'B0002VNKPO',
       'B0002VNKPY', 'B0002VNKQS', 'B0002VNKRM', 'B0002VNKSG', 'B0002VQ0WO'],
      dtype='object', name='ProductId', length=3091)

In [64]:
final_rating = filtered_rating[filtered_rating['ProductId'].isin(famous_resto)]

In [65]:
final_rating.drop_duplicates()

,UserId,ProductId,Rating
4,A3IAAVS479H7M7,0737104473,1.0
5,AKJHHD5VEH7VG,0762451459,5.0
6,A1BG8QW55XHN6U,1304139212,5.0
7,A22VW0P4VZHDE3,1304139220,5.0
10,A1BG8QW55XHN6U,130414643X,5.0
...,...,...,...
99742,A2P5CDZF56NLVL,B0002VNKRM,5.0
99746,A1ASSKIYJ4Z9K8,B0002VNKSG,5.0
99828,A6WV0OPKDYP2F,B0002VQ0WO,4.0
99848,A70G84Z95RU3N,B0002VQ0WO,5.0


In [66]:
#Plot the pivot table with product Id as index and user ID as columns names
pt = final_rating.pivot_table(index='ProductId',columns='UserId',values='Rating')
pt

UserId,A00473363TJ8YSZ3YAGG9,A0733801U845JJGV4H8V,A0908131Z7BWYSMRQ16T,A099766128UI0NCS98N1E,A1008GFLTBL76H,A100V5X97DUOXV,A100VQNP6I54HS,A1016DX7QKYBDC,A103D23F4SGCJY,A103U3KR4L2ZXT,...,AZTX0RHQ1O9CB,AZTX1196S2WR9,AZU3L6HCEFG7B,AZUI6YY673GW5,AZVYUXWLR63AZ,AZW9UD7Z7J0WL,AZWXG6KBXXC2N,AZZHB6U54UDYW,AZZNK89PXD006,AZZQXL8VDCFTV
ProductId,,,,,,,,,,,,,,,,,,,,,
0737104473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0762451459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1304139212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1304139220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130414643X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0002VNKPY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0002VNKQS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0002VNKRM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
# fill missing values with zeros
pt.fillna(0,inplace=True)
pt

UserId,A00473363TJ8YSZ3YAGG9,A0733801U845JJGV4H8V,A0908131Z7BWYSMRQ16T,A099766128UI0NCS98N1E,A1008GFLTBL76H,A100V5X97DUOXV,A100VQNP6I54HS,A1016DX7QKYBDC,A103D23F4SGCJY,A103U3KR4L2ZXT,...,AZTX0RHQ1O9CB,AZTX1196S2WR9,AZU3L6HCEFG7B,AZUI6YY673GW5,AZVYUXWLR63AZ,AZW9UD7Z7J0WL,AZWXG6KBXXC2N,AZZHB6U54UDYW,AZZNK89PXD006,AZZQXL8VDCFTV
ProductId,,,,,,,,,,,,,,,,,,,,,
0737104473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0762451459,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1304139212,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1304139220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
130414643X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0002VNKPY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B0002VNKQS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B0002VNKRM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
# measure using cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
similarity_score = cosine_similarity(pt)
df2 = pd.DataFrame(similarity_score)
df2

,0,1,2,3,4,5,6,7,8,9,...,3081,3082,3083,3084,3085,3086,3087,3088,3089,3090
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.707107,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.707107,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.707107,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3086,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.131306,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3088,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [69]:
# measure using euclidean distance
from sklearn.metrics.pairwise import euclidean_distances
similarity_score_2 = euclidean_distances(pt)
df3 = pd.DataFrame(similarity_score_2)
df3

,0,1,2,3,4,5,6,7,8,9,...,3081,3082,3083,3084,3085,3086,3087,3088,3089,3090
0,0.000000,5.099020,5.099020,5.099020,5.099020,5.099020,5.099020,7.141428,5.099020,7.141428,...,7.681146,3.162278,12.845233,4.123106,4.123106,5.000000,1.414214,11.313708,5.099020,8.185353
1,5.099020,0.000000,7.071068,7.071068,7.071068,7.071068,7.071068,8.660254,7.071068,8.660254,...,9.110434,5.830952,13.747727,6.403124,6.403124,7.000000,5.099020,12.328828,7.071068,9.539392
2,5.099020,7.071068,0.000000,7.071068,0.000000,7.071068,7.071068,8.660254,7.071068,5.000000,...,9.110434,5.830952,13.747727,6.403124,6.403124,7.000000,5.099020,12.328828,7.071068,9.539392
3,5.099020,7.071068,7.071068,0.000000,7.071068,7.071068,0.000000,8.660254,0.000000,5.000000,...,9.110434,5.830952,13.747727,6.403124,6.403124,7.000000,5.099020,12.328828,7.071068,9.539392
4,5.099020,7.071068,0.000000,7.071068,0.000000,7.071068,7.071068,8.660254,7.071068,5.000000,...,9.110434,5.830952,13.747727,6.403124,6.403124,7.000000,5.099020,12.328828,7.071068,9.539392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3086,5.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,8.602325,7.000000,8.602325,...,9.055385,5.744563,13.711309,6.324555,6.324555,0.000000,5.000000,12.288206,7.000000,9.486833
3087,1.414214,5.099020,5.099020,5.099020,5.099020,5.099020,5.099020,7.141428,5.099020,7.141428,...,7.549834,3.162278,12.845233,4.123106,4.123106,5.000000,0.000000,11.313708,5.099020,8.185353
3088,11.313708,12.328828,12.328828,12.328828,12.328828,12.328828,12.328828,13.304135,12.328828,13.304135,...,13.601471,11.661904,17.058722,11.958261,11.958261,12.288206,11.313708,0.000000,12.328828,13.892444
3089,5.099020,7.071068,7.071068,7.071068,7.071068,7.071068,7.071068,8.660254,7.071068,8.660254,...,9.110434,5.830952,13.747727,6.403124,6.403124,7.000000,5.099020,12.328828,0.000000,9.539392


In [70]:
#Recommend a Product

**Collaborative**

In [81]:
data=df_copy.copy()

In [82]:
#check details
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   UserId     100000 non-null  object 
 1   ProductId  100000 non-null  object 
 2   Rating     100000 non-null  float64
dtypes: float64(1), object(2)
memory usage: 2.3+ MB


In [83]:
# check null
print(data.isnull().sum())

UserId       0
ProductId    0
Rating       0
dtype: int64


In [84]:
data

,UserId,ProductId,Rating
0,A39HTATAQ9V7YF,0205616461,5.0
1,A3JM6GV9MNOF9X,0558925278,3.0
2,A1Z513UWSAAO0F,0558925278,5.0
3,A1WMRR494NWEWV,0733001998,4.0
4,A3IAAVS479H7M7,0737104473,1.0
...,...,...,...
99995,A3NCLOCXRJT30Z,B0002VQ0WO,3.0
99996,A1LALJTW2NFBH8,B0002VQ0WO,5.0
99997,A288BYAMYVWCI1,B0002VQ0WO,4.0
99998,A3VH23K0SNNOG9,B0002VQ0WO,2.0


In [85]:
data=data.iloc[5000:10000]

In [86]:
ratings = data.pivot(index = 'UserId', columns = 'ProductId', values = 'Rating').fillna(0)
ratings.head()

ProductId,B000052XZP,B000052XZQ,B000052XZX,B000052Y0B,B000052Y0C,B000052Y0I,B000052Y0S,B000052Y0U,B000052Y0Y,B000052Y1M,...,B000052YPE,B000052YPF,B000052YPG,B000052YPH,B000052YPM,B000052YPU,B000052YPV,B000052YPY,B000052YQ0,B000052YQ2
UserId,,,,,,,,,,,,,,,,,,,,,
A07410232KYRFR25CIUGJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0870685EP51H0X8LFFU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A100UD67AHFODS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1034GEUA8ZVFR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A103BJIOJSDJL1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
import numpy as np
ratings['user_index'] = np.arange(0, ratings.shape[0],1)
ratings.set_index(['user_index'], inplace = True)
# ratings=ratings.iloc[:5000]

In [88]:
# check sparsity
sparsity = (1 - data.shape[0]/(ratings.shape[0]*ratings.shape[1]))*100
print('sparsity : {:.2f}%'.format(sparsity))

sparsity : 99.22%


In [90]:
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
#SVD
s_ratings = csr_matrix(ratings)

u,s,vT = svds(s_ratings,k = 10)

In [91]:
#Construct diagonal array in SVD
sigma = np.diag(s)

print("Shape of U : {}".format(u.shape))
print("Shape of Sigma, S : {}".format(sigma.shape))
print("Shape of VT : {}".format(vT.shape))

Shape of U : (4829, 10)
Shape of Sigma, S : (10, 10)
Shape of VT : (10, 132)


In [92]:
#Reconstruct ratings with predicted values using SVD
ratings_pred = np.matmul(np.matmul(u,sigma),vT)
pred_data = pd.DataFrame(ratings_pred, columns = ratings.columns, index = ratings.index)
pred_data.head()

ProductId,B000052XZP,B000052XZQ,B000052XZX,B000052Y0B,B000052Y0C,B000052Y0I,B000052Y0S,B000052Y0U,B000052Y0Y,B000052Y1M,...,B000052YPE,B000052YPF,B000052YPG,B000052YPH,B000052YPM,B000052YPU,B000052YPV,B000052YPY,B000052YQ0,B000052YQ2
user_index,,,,,,,,,,,,,,,,,,,,,
0,3.033976e-04,1.853767e-17,-8.195614e-03,6.806779e-17,-1.192763e-16,-1.999997e-16,-1.307724e-17,-1.483950e-16,-2.362224e-16,-3.271048e-07,...,5.798898e-17,1.257424e-16,-4.282775e-17,2.589724e-18,-1.284083e-16,5.198669e-17,-1.517822e-07,1.325853e-02,8.816854e-09,1.378535e-02
1,-2.974919e-03,6.706356e-17,7.661050e-02,4.244091e-16,-4.247885e-16,-7.735136e-16,-2.151856e-16,-5.128488e-16,-3.068929e-16,4.141087e-06,...,3.430276e-16,5.311627e-16,-3.252641e-16,2.047764e-17,-4.897646e-16,2.967855e-16,1.839731e-06,-2.483429e-03,-2.857540e-08,6.504984e-02
2,-5.914132e-16,3.347801e-32,-1.651670e-17,1.654152e-31,-2.179288e-31,-3.666980e-31,1.512536e-31,-2.524450e-31,-3.454062e-31,-1.820046e-22,...,1.307775e-31,2.220680e-31,-1.324616e-31,6.512896e-33,-2.277875e-31,1.133354e-31,-8.788310e-23,8.266045e-18,-2.453942e-21,1.424879e-17
3,4.114542e-07,4.085557e-18,3.435816e-07,2.556650e-17,-4.971587e-17,-4.728647e-17,1.201722e-16,-5.982160e-17,-5.023420e-17,-2.415039e-08,...,2.474873e-17,4.782840e-17,-1.789757e-17,1.416027e-19,-5.709771e-17,2.051939e-17,-1.121469e-08,-1.459457e-07,5.444855e-09,-4.562047e-05
4,-6.194203e-04,7.256941e-17,1.289500e-02,1.561677e-16,-2.616577e-16,-8.234777e-16,5.610046e-16,-3.340512e-16,-6.714732e-16,1.526835e-07,...,3.132470e-16,4.451884e-16,-1.311688e-16,3.567291e-18,-3.857435e-16,2.686587e-16,6.398933e-08,-1.595069e-05,-3.662377e-07,1.278757e-02


In [93]:
# # Check MSE
a = ratings.values.reshape(-1)
b = ratings_pred.reshape(-1)
mse = np.sum((a-b)**2)/a.shape[0]
print("MSE : {:.4f}".format(mse))

MSE : 0.0806
